# Import libraries and dataset

In [ ]:
import pandas as pd
from datasets import Dataset
from dawid_skene_model import list2array, DawidSkeneModel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from langchain.schema import HumanMessage, SystemMessage
from tqdm.notebook import tqdm
import numpy as np
from collections import Counter
import os
import google.generativeai as genai
from huggingface_hub import login
import os

In [ ]:
# Esegui l'autenticazione
login(token=os.getenv("HUGGINGFACE_TOKEN"))

In [ ]:
from datasets import load_dataset

# Carica il dataset Chatbot Arena Conversations
chatbot_arena = load_dataset("lmsys/chatbot_arena_conversations")
display(chatbot_arena)

# Analyze the dataset for italian conversations

In [ ]:
# Converte in DataFrame per un'analisi più semplice
df = pd.DataFrame(chatbot_arena["train"])

# Filtra solo le conversazioni in italiano
df_italian = df[df["language"] == "Italian"]

# Mostra alcune righe per conferma
display(df_italian.head())

# Numero di conversazioni in italiano
print(f"Numero totale di conversazioni in italiano: {len(df_italian)}")


In [ ]:
# Conta quante volte ogni modello è stato dichiarato vincitore
winner_counts = df_italian["winner"].value_counts()

print("\nDistribuzione delle vittorie per modello:")
print(winner_counts)

# Visualizzazione con grafico a barre
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.barplot(x=winner_counts.index, y=winner_counts.values, palette="viridis")
plt.xticks(rotation=45)
plt.title("Numero di vittorie per ciascun modello (italiano)")
plt.xlabel("Modelli")
plt.ylabel("Numero di vittorie")
plt.show()

In [ ]:
# Conta quante volte ogni modello è stato dichiarato vincitore
turn_counts = df_italian["turn"].value_counts()

print("\nDistribuzione del numero di turni per le conversazioni:")
print(turn_counts)

# Visualizzazione con grafico a barre
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.barplot(x=turn_counts.index, y=turn_counts.values, palette="viridis")
plt.xticks(rotation=45)
plt.title("Numero di turn per ciascuna conversazione (italiano)")
plt.xlabel("Modelli")
plt.ylabel("Numero di turn")
plt.show()

In [ ]:
# Filtra solo le conversazioni con turn == 1
df_italian_turn = df_italian[df_italian["turn"] == 1]
df_italian_turn = df_italian_turn.drop(columns=["turn", "anony", "language", "tstamp", "openai_moderation", "toxic_chat_tag", "judge"])

#  Reset dell'indice del DataFrame dopo il filtraggio
df_italian_turn = df_italian_turn.reset_index(drop=True)

# Mostra il numero di conversazioni rimaste
print(f"Numero di conversazioni con un solo turno: {len(df_italian_turn)}")

# Visualizza alcune righe per conferma
display(df_italian_turn.head())

In [ ]:
from pprint import pprint
pprint(df_italian_turn['conversation_a'][11])
pprint(df_italian_turn['conversation_b'][11])

# Analyze dataset for english conversations

In [ ]:
# Converte in DataFrame per un'analisi più semplice
df = pd.DataFrame(chatbot_arena["train"])

# Filtra solo le conversazioni in inglese
df_eng = df[df["language"] == "English"] 

# Filtra solo le conversazioni con turn == 1
df_eng = df_eng[df_eng["turn"] == 1]

df_eng = df_eng[(df_eng["winner"] == "model_a") | (df_eng["winner"] == "model_b")]

df_eng = df_eng.drop(columns=["turn", "anony", "language", "tstamp", "openai_moderation", "toxic_chat_tag", "judge"])

# Mostra alcune righe per conferma
display(df_eng.head())

# Numero di conversazioni in inglese
print(f"Numero totale di conversazioni in inglese con i filtri applicati: {len(df_eng)}")

In [ ]:
# Visualizzazione con grafico a barre
import matplotlib.pyplot as plt
import seaborn as sns

# Conta quante volte ogni modello è stato dichiarato vincitore
turn_counts = df_eng["model_a"].value_counts()

plt.figure(figsize=(12, 6))
sns.barplot(x=turn_counts.index, y=turn_counts.values, hue=turn_counts.index, dodge=False)
plt.xticks(rotation=45)
plt.title("Numero di turn per ciascuna conversazione (italiano)")
plt.xlabel("Modelli")
plt.ylabel("Numero di turn")
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Creiamo una colonna con le coppie di modelli ordinate
df_eng["model_pair"] = df_eng.apply(lambda row: tuple(sorted([row["model_a"], row["model_b"]])), axis=1)

# Conta il numero totale di sfide per ogni coppia di modelli
model_pair_total = df_eng["model_pair"].value_counts().reset_index()
model_pair_total.columns = ["model_pair", "Total Matches"]

# Conta le vittorie di ciascun modello all'interno di ogni coppia
model_wins = df_eng.groupby(["model_pair", "winner"]).size().reset_index(name="Wins")

# Separiamo le vittorie di ciascun modello
model_wins_a = model_wins[model_wins["winner"] == "model_a"].drop(columns=["winner"]).rename(columns={"Wins": "Wins_A"})
model_wins_b = model_wins[model_wins["winner"] == "model_b"].drop(columns=["winner"]).rename(columns={"Wins": "Wins_B"})

# Merge per ottenere il totale e le vittorie di ciascun modello
model_stats = (
    model_pair_total
    .merge(model_wins_a, on="model_pair", how="left")
    .merge(model_wins_b, on="model_pair", how="left")
)

# Sostituiamo i NaN con 0 in modo sicuro usando `.fillna()` senza inplace=True
model_stats = model_stats.assign(
    Wins_A=model_stats["Wins_A"].fillna(0),
    Wins_B=model_stats["Wins_B"].fillna(0)
)

# Calcoliamo le percentuali di vittoria
model_stats["Win_A (%)"] = (model_stats["Wins_A"] / model_stats["Total Matches"]) * 100
model_stats["Win_B (%)"] = (model_stats["Wins_B"] / model_stats["Total Matches"]) * 100

# Ordiniamo il DataFrame per il numero totale di partite giocate
model_stats = model_stats.sort_values(by="Total Matches", ascending=False)

# Mostriamo le prime 10 coppie più frequenti con percentuale di vittoria
print("\nTop 10 coppie di modelli con percentuali di vittoria:")
display(model_stats.head(10))

In [ ]:
df_eng = df_eng[((df_eng["model_a"] == "vicuna-13b") & (df_eng["model_b"] == "koala-13b")) |
                ((df_eng["model_a"] == "koala-13b") & (df_eng["model_b"] == "vicuna-13b"))]

# Reset index
df_eng = df_eng.reset_index(drop=True)

# Mostra alcune righe per conferma
display(df_eng.head())

# Numero di conversazioni in inglese
print(f"Numero totale di conversazioni in inglese con i filtri applicati: {len(df_eng)}")

In [ ]:
from pprint import pprint
pprint(df_eng['conversation_a'][11])
pprint(df_eng['conversation_b'][11])

In [ ]:
df_eng['conversation_a'][0][0]['content'] #question

In [ ]:
df_eng['conversation_a'][0][1]['content'] #answer

In [ ]:
# Extract 'content' with role 'user' as question and the one with role 'assistant' as answer for conversation 'a' and 'b'
# Save question and answer in a dataframe
qa = pd.DataFrame(columns=['question', 'answer_a'])
idx = 0

for elem in df_eng['conversation_a']:
    if elem[0]['role'] == 'user':
        qa.loc[idx] = [elem[0]['content'], elem[1]['content']]
        idx = idx + 1

display(qa)

In [ ]:
# Add answer_b for the conversation_b of the dataset
qa['answer_b'] = df_eng['conversation_b'].apply(lambda x: x[1]['content'])
qa['winner'] = df_eng['winner']
display(qa)

# Create big model reference

In [ ]:
from huggingface_hub import InferenceClient
import json

token_pro = os.getenv('HUGGINGFACE_TOKEN')
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
    token=token_pro
)

def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        model=repo_id,  # Explicitly specify the model
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 1028},
            "task": "text-generation",
        },
    )
    return json.loads(response.decode())[0]["generated_text"].strip()

In [ ]:
import re
import pandas as pd
from tqdm import tqdm

# Register tqdm with pandas
tqdm.pandas()

def evaluate_pair(model, row):
    prompt = f"""
    You are a critic evaluating two different answers to the same question on any subject.
    Compare the two answers based on:
    1. **Completeness**: Does the answer fully clarify the question?
    2. **Relevance**: Is the answer relevant to the question?
    3. **Correctness**: Is the information in the answer accurate?
    4. **Clarity**: Is the answer clearly understandable by a human reader?
    
    Evaluate the following question and its two answers, and determine which is better, more correct, and understandable by a human reader.
    Respond with 'model_a' if the first answer is better, otherwise respond with 'model_b'.

    Give the response exactly in the following format, and nothing else:

    Output:::
    Best model: [model_a/model_b]
    Output:::

    Now, consider the following question and its two answers:
    **Question:** {row['question']}
    **Answer 1:** {row['answer_a']}
    **Answer 2:** {row['answer_b']}

    Output:::
    """

    if isinstance(model, InferenceClient):
        response = call_llm(model, prompt)

        # Extract the best model from the response
        match = re.search(r"Best model:\s*(model_a|model_b)", response)
        return match.group(1) if match else "Unknown"  # Default to "Unknown" if parsing fails
    else:
        response = model.generate(row['question'], row['answer_a'], row['answer_b'])
        return response

In [ ]:
# ** Valutazione con Mixtral **
qa["Mixtral"] = qa.progress_apply(lambda row: evaluate_pair(llm_client, row), axis=1)

# ** Confronto tra GPT e Gemini **
qa["Agreement Mixtral-Groundtruth"] = qa["Mixtral"] == qa["winner"]

In [ ]:
display(qa['Mixtral'].value_counts())

In [ ]:
# Delete all the rows of the dataframe 'qa' that have the value 'Unknown' in the column 'Mixtral'
qa = qa[qa['Mixtral'] != 'Unknown']
display(qa)

In [ ]:
# Reset qa index
qa = qa.reset_index(drop=True)
display(qa)

In [ ]:
# Percentage of Mixtral correct judgment
mixtral_vs_groundtruth = qa["Agreement Mixtral-Groundtruth"].mean() * 100
print("Correct judgment Mixtral:", mixtral_vs_groundtruth)

# Convertiamo le risposte in valori numerici per la heatmap
heatmap_df = qa[[
    "winner", 
    "Mixtral"
]].map(lambda x: 1 if x == "model_a" else 0)

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_df, annot=True, fmt="d", cmap="coolwarm", cbar=True)
plt.title("Compare Mixtral judgment with the groundtruth")
plt.xlabel("Mixtral model")
plt.ylabel("Groundtruth")
plt.show()

In [ ]:
# Save the dataframe to a CSV file
qa.to_csv("faq_evaluation_results_with_Mixtral.csv", index=False, encoding="utf-8")

# Create models - Ensemble

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ** Ensemble Model Configuration **
MODEL_NAMES = {
    "DistilBERT": "distilbert/distilbert-base-uncased", 
    "MiniSBERT": "sentence-transformers/all-MiniLM-L6-v2",
    "PalmBonsai": "Writer/palmyra-small", 
    "MiniGptBased": "ComCom/gpt2-small"
}

class HuggingFaceModel:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = token_pro)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, use_auth_token = token_pro)
        
        if self.tokenizer.pad_token is None:
            self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
            self.model.resize_token_embeddings(len(self.tokenizer))

    def generate(self, question, answer1, answer2):
        inputs1 = self.tokenizer(f"Question: {question} Answer: {answer1}", return_tensors="pt", truncation=True, max_length=512, padding="max_length")
        inputs2 = self.tokenizer(f"Question: {question} Answer: {answer2}", return_tensors="pt", truncation=True, max_length=512, padding="max_length")
        
        with torch.no_grad():
            logits1 = self.model(**inputs1).logits
            logits2 = self.model(**inputs2).logits
        
        score1 = torch.softmax(logits1, dim=1)[0][1].item()
        score2 = torch.softmax(logits2, dim=1)[0][1].item()
        
        return "model_a" if score1 > score2 else "model_b"

# ** Initialize Ensemble Models **
ensemble_models = {name: HuggingFaceModel(model) for name, model in MODEL_NAMES.items()}

In [ ]:
# ** Enable progress tracking **
tqdm.pandas()

# ** Evaluation with ensemble models **
for model_name, model in ensemble_models.items():
    qa[model_name] = qa.progress_apply(lambda row: evaluate_pair(model, row), axis=1)

# ** Majority Voting Function **
def majority_voting(row):
    votes = [row[model] for model in MODEL_NAMES.keys()]
    return Counter(votes).most_common(1)[0][0]  # Most voted option

# ** Compute Majority Voting **
qa["Majority Voting"] = qa.apply(majority_voting, axis=1)

In [ ]:
# ** Compare with groundtruth **
qa["Agreement Majority-Mixtral"] = qa["Majority Voting"] == qa["Mixtral"]

# ** Compute Agreement Percentage **
majority_vs_mixtral = qa["Agreement Majority-Mixtral"].mean() * 100

# ** Encode Answers for Consensus Analysis **
def encode_answers(qa):
    return qa.replace({"model_a": 1, "model_b": 0})

In [ ]:
display(qa)

In [ ]:
encoded_df = encode_answers(qa.iloc[:, 3:-1]).apply(pd.to_numeric, errors="coerce")
display(encoded_df)

In [ ]:
# Definisci le colonne dei modelli
model_columns = ["DistilBERT", "MiniSBERT", "PalmBonsai", "MiniGptBased"]

# FUNZIONI AUSILIARIE
# Converti il DataFrame in una lista nidificata (dataset_list)
def dataframe_to_dataset_list(df, model_columns):
    dataset_list = []
    for _, row in df.iterrows():
        task = []
        for model in model_columns:
            response = row[model]  # Prendi la risposta del modello
            task.append([0] if response == "model_b" else [1])  # Converti in formato numerico
        dataset_list.append(task)
    return dataset_list

# Converti il DataFrame in dataset_list
dataset_list = dataframe_to_dataset_list(qa, model_columns)

# Converti in tensore NumPy
class_num = 2  # Solo due classi: Sì (1) e No (0)
dataset_tensor = list2array(class_num, dataset_list)

# Inizializza e lancia il modello di Dawid & Skene
model = DawidSkeneModel(class_num=2, max_iter=40, tolerance=1e-5)
marginal_predict, error_rates, worker_reliability, predict_label = model.run(dataset_tensor)

# Converti le predizioni finali in "Sì" o "No"
final_answers = ["model_a" if p[1] > 0.5 else "model_b" for p in predict_label]

# Aggiungi i risultati al DataFrame
qa["Dawid & Skene Multi-Class"] = final_answers

In [ ]:
# Mostra il confronto con Majority Voting e Gemini
qa["Agreement D&S-Mixtral"] = qa["Dawid & Skene Multi-Class"] == qa["Mixtral"]

# Calcola le percentuali di accordo
ds_multi_vs_gemini = qa["Agreement D&S-Mixtral"].mean() * 100

summary_df = pd.DataFrame({
    "Metodo": ["Majority Voting", "Dawid & Skene Multi-Class", ],
    "Concordanza con Groundtruth (%)": [majority_vs_mixtral, ds_multi_vs_gemini]
})

display(summary_df)
import seaborn as sns
import matplotlib.pyplot as plt

# Convertiamo le risposte in valori numerici per la heatmap
heatmap_df = qa[[
    "Majority Voting", 
    "Dawid & Skene Multi-Class"
]].map(lambda x: 1 if x == "model_a" else 0)

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_df, annot=True, fmt="d", cmap="coolwarm", cbar=True)
plt.title("Confronto tra Majority Voting e Dawid & Skene")
plt.xlabel("Metodo di Ensemble o Modello")
plt.ylabel("Domande")
plt.show()

# Compare the results with the original groundtruth

In [ ]:
display(qa)

In [ ]:
# Mostra il confronto con Majority Voting e Gemini
qa["Agreement D&S-Groundtruth"] = qa["Dawid & Skene Multi-Class"] == qa["winner"]
qa["Agreement Majority-Groundtruth"] = qa["Majority Voting"] == qa["winner"]

# Calcola le percentuali di accordo
ds_truth = qa["Agreement D&S-Groundtruth"].mean() * 100
majority_truth = qa["Agreement Majority-Groundtruth"].mean() * 100

summary_df = pd.DataFrame({
    "Metodo": ["Majority Voting", "Dawid & Skene Multi-Class", "Mixtral"],
    "Concordanza con Groundtruth (%)": [majority_truth, ds_truth, mixtral_vs_groundtruth]
})

display(summary_df)

In [ ]:
# **Distribuzione delle risposte per ciascun modello**
print("\nDistribuzione delle risposte per ciascun modello:")
for model in model_columns:
    print(f"{model}:")
    print(qa[model].value_counts(normalize=True) * 100, "\n")

# **Distribuzione delle risposte per Mixtral**
print("\nDistribuzione delle risposte di Mixtral:")
print(qa["Mixtral"].value_counts(normalize=True) * 100)

# PoLL with bigger models to improve the results of Mixtral

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ** Ensemble Model Configuration **
MODEL_NAMES_NEW = {
    "GPT": "openai-community/gpt2",
    "DistilBERT": "distilbert/distilbert-base-uncased",
    "MiniSBERT": "sentence-transformers/all-MiniLM-L6-v2", 
    "PalmBonsai": "Writer/palmyra-small", 
    "MiniGptBased": "ComCom/gpt2-small", 
    "DeepHaiku": "fabianmmueller/deep-haiku-gpt-2", 
    #"Llama": "meta-llama/Llama-3.2-1B"
}

class HuggingFaceModel:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = token_pro)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, use_auth_token = token_pro)
        
        if self.tokenizer.pad_token is None:
            self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
            self.model.resize_token_embeddings(len(self.tokenizer))

    def generate(self, question, answer1, answer2):
        inputs1 = self.tokenizer(f"Question: {question} Answer: {answer1}", return_tensors="pt", truncation=True, max_length=512, padding="max_length")
        inputs2 = self.tokenizer(f"Question: {question} Answer: {answer2}", return_tensors="pt", truncation=True, max_length=512, padding="max_length")
        
        with torch.no_grad():
            logits1 = self.model(**inputs1).logits
            logits2 = self.model(**inputs2).logits
        
        score1 = torch.softmax(logits1, dim=1)[0][1].item()
        score2 = torch.softmax(logits2, dim=1)[0][1].item()
        
        return "model_a" if score1 > score2 else "model_b"

# ** Initialize Ensemble Models **
ensemble_models = {name: HuggingFaceModel(model) for name, model in MODEL_NAMES_NEW.items()}

In [ ]:
# load csv file "faq_evaluation_results_with_Mixtral.csv"
qa = pd.read_csv("faq_evaluation_results_with_Mixtral.csv")

In [ ]:
display(qa)

In [ ]:
# ** Enable progress tracking **
tqdm.pandas()

# ** Evaluation with ensemble models **
for model_name, model in ensemble_models.items():
    qa[model_name] = qa.progress_apply(lambda row: evaluate_pair(model, row), axis=1)

In [ ]:
# ** Majority Voting Function **
models = ['GPT', 'DistilBERT', 'Mixtral', 'PalmBonsai', 'MiniGptBased']
def majority_voting(row, models = models):
    votes = [row[model] for model in models]
    return Counter(votes).most_common(1)[0][0]  # Most voted option

# ** Compute Majority Voting **
qa["Majority Voting"] = qa.apply(majority_voting, axis=1)

In [ ]:
# Definisci le colonne dei modelli
model_columns = models

# FUNZIONI AUSILIARIE
# Converti il DataFrame in una lista nidificata (dataset_list)
def dataframe_to_dataset_list(df, model_columns):
    dataset_list = []
    for _, row in df.iterrows():
        task = []
        for model in model_columns:
            response = row[model]  # Prendi la risposta del modello
            task.append([0] if response == "model_b" else [1])  # Converti in formato numerico
        dataset_list.append(task)
    return dataset_list

# Converti il DataFrame in dataset_list
dataset_list = dataframe_to_dataset_list(qa, model_columns)

# Converti in tensore NumPy
class_num = 2  # Solo due classi: Sì (1) e No (0)
dataset_tensor = list2array(class_num, dataset_list)

# Inizializza e lancia il modello di Dawid & Skene
model = DawidSkeneModel(class_num=2, max_iter=40, tolerance=1e-5)
marginal_predict, error_rates, worker_reliability, predict_label = model.run(dataset_tensor)

# Converti le predizioni finali in "Sì" o "No"
final_answers = ["model_a" if p[1] > 0.5 else "model_b" for p in predict_label]

# Aggiungi i risultati al DataFrame
qa["Dawid & Skene Multi-Class"] = final_answers

In [ ]:
display(qa)

In [ ]:
# save csv file with all results
qa.to_csv("faq_evaluation_results_with_Mixtral_and_Ensemble.csv", index=False, encoding="utf-8")

# Evaluate results

In [ ]:
# load csv file
qa = pd.read_csv("faq_evaluation_results_with_Mixtral_and_Ensemble.csv")
display(qa)

In [ ]:
# Percentage of Mixtral correct judgment
mixtral_vs_groundtruth = qa["Agreement Mixtral-Groundtruth"].mean() * 100
print("Correct judgment Mixtral:", mixtral_vs_groundtruth)

# Convertiamo le risposte in valori numerici per la heatmap
heatmap_df = qa[[
    "winner", 
    "Mixtral"
]].map(lambda x: 1 if x == "model_a" else 0)

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_df, annot=True, fmt="d", cmap="coolwarm", cbar=True)
plt.title("Compare Mixtral judgment with the groundtruth")
plt.xlabel("Mixtral model")
plt.ylabel("Groundtruth")
plt.show()

In [ ]:
# Mostra il confronto con Majority Voting e Gemini
qa["Agreement D&S-Groundtruth"] = qa["Dawid & Skene Multi-Class"] == qa["winner"]
qa["Agreement Majority-Groundtruth"] = qa["Majority Voting"] == qa["winner"]

# Calcola le percentuali di accordo
ds_truth = qa["Agreement D&S-Groundtruth"].mean() * 100
majority_truth = qa["Agreement Majority-Groundtruth"].mean() * 100

summary_df = pd.DataFrame({
    "Metodo": ["Majority Voting", "Dawid & Skene Multi-Class", "Mixtral"],
    "Concordanza con Groundtruth (%)": [majority_truth, ds_truth, mixtral_vs_groundtruth]
})

display(summary_df)

In [ ]:
# **Distribuzione delle risposte per ciascun modello**
print("\nDistribuzione delle risposte per ciascun modello:")
for model in model_columns:
    print(f"{model}:")
    print(qa[model].value_counts(normalize=True) * 100, "\n")

In [ ]:
# ** Concordanza di ciascun modello con il groundtruth **
print("\nConcordanza di ciascun modello con il groundtruth:")
for model in model_columns:
    print(f"{model}:")
    qa[f"Agreement {model}-Groundtruth"] = qa[model] == qa["winner"]
    print(qa[f"Agreement {model}-Groundtruth"].mean() * 100, "\n")

# Analyze models correctness

In [ ]:
# load csv file "faq_evaluation_results_with_Mixtral.csv"
qa = pd.read_csv("updated_faq.csv")
display(qa)

In [ ]:
# ** Concordanza di ciascun modello con il groundtruth **
model_columns = ["Mixtral", "GPT", "DistilBERT", "MiniSBERT", "PalmBonsai", "MiniGptBased", "DeepHaiku", "Llama", "Gemma"]
print("\nConcordanza di ciascun modello con il groundtruth:")
for model in model_columns:
    print(f"{model}:")
    qa[f"Agreement {model}-Groundtruth"] = qa[model] == qa["winner"]
    print(qa[f"Agreement {model}-Groundtruth"].mean() * 100, "\n")

In [ ]:
# ** Majority Voting Function **
models = ["Mixtral", "GPT", "DistilBERT", "MiniSBERT", "PalmBonsai", "MiniGptBased", "DeepHaiku", "Llama", "Gemma"]
def majority_voting(row, models = models):
    votes = [row[model] for model in models]
    return Counter(votes).most_common(1)[0][0]  # Most voted option

# ** Compute Majority Voting **
qa["Majority Voting"] = qa.apply(majority_voting, axis=1)

In [ ]:
# Definisci le colonne dei modelli
model_columns = models

# FUNZIONI AUSILIARIE
# Converti il DataFrame in una lista nidificata (dataset_list)
def dataframe_to_dataset_list(df, model_columns):
    dataset_list = []
    for _, row in df.iterrows():
        task = []
        for model in model_columns:
            response = row[model]  # Prendi la risposta del modello
            task.append([0] if response == "model_b" else [1])  # Converti in formato numerico
        dataset_list.append(task)
    return dataset_list

# Converti il DataFrame in dataset_list
dataset_list = dataframe_to_dataset_list(qa, model_columns)

# Converti in tensore NumPy
class_num = 2  # Solo due classi: Sì (1) e No (0)
dataset_tensor = list2array(class_num, dataset_list)

# Inizializza e lancia il modello di Dawid & Skene
model = DawidSkeneModel(class_num=2, max_iter=40, tolerance=1e-5)
marginal_predict, error_rates, worker_reliability, predict_label = model.run(dataset_tensor)

# Converti le predizioni finali in "Sì" o "No"
final_answers = ["model_a" if p[1] > 0.5 else "model_b" for p in predict_label]

# Aggiungi i risultati al DataFrame
qa["Dawid & Skene Multi-Class"] = final_answers

In [ ]:
# Percentage of Mixtral correct judgment
mixtral_vs_groundtruth = qa["Agreement Mixtral-Groundtruth"].mean() * 100
print("Correct judgment Mixtral:", mixtral_vs_groundtruth)

In [ ]:
# Mostra il confronto con Majority Voting e Gemini
qa["Agreement D&S-Groundtruth"] = qa["Dawid & Skene Multi-Class"] == qa["winner"]
qa["Agreement Majority-Groundtruth"] = qa["Majority Voting"] == qa["winner"]

# Calcola le percentuali di accordo
ds_truth = qa["Agreement D&S-Groundtruth"].mean() * 100
majority_truth = qa["Agreement Majority-Groundtruth"].mean() * 100

summary_df = pd.DataFrame({
    "Metodo": ["Majority Voting", "Dawid & Skene Multi-Class", "Mixtral"],
    "Concordanza con Groundtruth (%)": [majority_truth, ds_truth, mixtral_vs_groundtruth]
})

display(summary_df)